Cambiar LR

In [ ]:
import os

import random
import gym
import pylab
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

#tf.config.experimental_run_functions_eagerly(True)
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import copy

EPISODES = 100000 # Number of times the enviroment is ran
LR = 0.0025 # Learning rate 
NUM_ACTIONS = 4 # Number of possible action in the environment
NUM_STATES = 8 # Number of possible states in the exvironment
EPOCHS = 10 # Epochs to train the network (recommended between 3 and 30)
BATCH_SIZE = 64 # Batch size for the neural nets
BUFFER_SIZE = 2048 # Buffer of experiences
SHUFFLE = True # Whether to shuffle data or not while training
OPTIMIZER = Adam # Optimizer for both actor and critic
GAMMA = 0.99 # Used for the estimated reward
NORMALIZE = True # Whether to normalize GAE or not


# Create the actor used to select the action given an state
class Actor_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Softmax as there are different probabilities depending on the action
        output = Dense(NUM_ACTIONS, activation="softmax")(X)
        
        # Compile the model with the custom loss
        self.model = Model(inputs = X_input, outputs = output)
        self.model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER(lr=LR))

# Create the critic which will criticise how the actor is performing    
class Critic_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Linear output to know how good the action is
        value = Dense(1)(X)
        
        # Compile it with mse loss and gradient descent
        self.model = Model(inputs=X_input, outputs = value)
        self.model.compile(loss='mse', optimizer=OPTIMIZER(lr=LR))

# Combine both Actor and Critic to create the agent
class PPOAgent:
    def __init__(self, env_name):
        # Environment parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.episode = 0 # used to track current number episoded since start
        self.max_average = 0 # record max average reached
        
        # Used to plot a grapgh of the train process
        self.scores_, self.average_ = [], []

        # Create Actor-Critic network models
        self.Actor = Actor_Model()
        self.Critic = Critic_Model()
    
        # Names for the models
        self.Actor_name = f"{self.env_name}_PPO_Actor.h5"
        self.Critic_name = f"{self.env_name}_PPO_Critic.h5"

    # Get the action given the current state    
    def act(self, state):
        # Use the network to predict the next action to take, using the model
        prediction = self.Actor.model.predict(state)[0]
        
        # Probability based to choose the action
        action = np.random.choice(NUM_ACTIONS, p=prediction)
        action_onehot = np.zeros([NUM_ACTIONS])
        action_onehot[action] = 1
        return action, action_onehot, prediction
    
    def discount_rewards(self, reward):#gaes is better
        # Compute the gamma-discounted rewards over an episode
        # We apply the discount and normalize it to avoid big variability of rewards
        running_add = 0
        discounted_r = np.zeros_like(reward)
        for i in reversed(range(0,len(reward))):
            running_add = running_add * GAMMA + reward[i]
            discounted_r[i] = running_add

        discounted_r -= np.mean(discounted_r) # normalizing the result
        discounted_r /= (np.std(discounted_r) + 1e-8) # divide by standard deviation
        return discounted_r
    
    def replay(self, states, actions, rewards, predictions, dones, next_states):
        # Reshape memory to appropriate shape for training
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        predictions = np.vstack(predictions)

        # Get Critic network predictions for state and next state
        values = self.Critic.model.predict(states)
        next_values = self.Critic.model.predict(next_states)

        # Compute de discounted reward
        discounted_rewards = self.discount_rewards(rewards)

        # Stack info to unpack it in the custom loss
        y_true = actions

        values = np.reshape(values, (np.shape(values)[0],))

        advantages = np.vstack(discounted_rewards - values)
        advantages = np.reshape(advantages, (np.shape(advantages)[0],))

        start_train = time.time()
        # Training Actor and Critic networks
        a_loss = self.Actor.model.fit(states, y_true, sample_weight=advantages, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        c_loss = self.Critic.model.fit(states, discounted_rewards, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        #print('Time: ', time.time()-start_train)
 
    def load(self):
        self.Actor.Actor.load_weights(self.Actor_name)
        self.Critic.Critic.load_weights(self.Critic_name)

    def save(self):
        self.Actor.model.save_weights(self.Actor_name)
        self.Critic.model.save_weights(self.Critic_name)

    def run_batch(self): # train every self.Training_batch episodes
        global LR
        state = self.env.reset()
        state = np.reshape(state, [1, NUM_STATES])
        done, score = False, 0
        finished = False
        while finished == False:
            # Instantiate or reset games memory
            states, next_states, actions, rewards, predictions, dones = [], [], [], [], [], []
            for t in range(BUFFER_SIZE):
                #self.env.render()
                # Actor picks an action
                action, action_onehot, prediction = self.act(state)
                # Retrieve new state, reward, and whether the state is terminal
                next_state, reward, done, _ = self.env.step(action)
                # Memorize (state, action, reward) for training
                states.append(state)
                next_states.append(np.reshape(next_state, [1, NUM_STATES]))
                actions.append(action_onehot)
                rewards.append(reward)
                dones.append(done)
                predictions.append(prediction)
                # Update current state
                state = np.reshape(next_state, [1, NUM_STATES])
                score += reward
                if done:
                    self.episode += 1
                    self.scores_.append(score)
                    #average, SAVING = self.PlotModel(score, self.episode)
                    #print("episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, EPISODES, score, average, SAVING))
                    if self.episode >= 100:
                        average = sum(self.scores_[-100:])/100
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, average))
                        if average > self.max_average:
                            self.max_average = average
                            if self.max_average > 150:
                                self.save()
                            LR *= 0.99
                            K.set_value(self.Actor.model.optimizer.learning_rate, LR)
                            K.set_value(self.Critic.model.optimizer.learning_rate, LR)
                            
                        if average > 200:
                            plt.plot(self.scores_)
                            plt.xlabel("Episode")
                            plt.ylabel("Score")
                            finished = True
                            break

                    else:
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, sum(self.scores_)/len(self.scores_)))
                    
                    
                    state, done, score = self.env.reset(), False, 0
                    state = np.reshape(state, [1, NUM_STATES])
                    
            self.replay(states, actions, rewards, predictions, dones, next_states)
            if self.episode >= EPISODES:
                break
        self.env.close()   
                    
if __name__ == "__main__":
    start = time.time()
    env_name = 'LunarLander-v2'
    agent = PPOAgent(env_name)
    #agent.run() # train as PPO, train every epesode
    agent.run_batch() # train as PPO, train every batch, trains better
    #agent.run_multiprocesses(num_worker = 8)  # train PPO multiprocessed (fastest)
    #agent.test()
    print((time.time() - start)/60)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Episode:     1		score: -200.78		average: -200.78
Episode:     2		score: -149.42		average: -175.10
Episode:     3		score:  -96.43		average: -148.88
Episode:     4		score: -195.99		average: -160.66
Episode:     5		score:  -82.28		average: -144.98
Episode:     6		score: -164.03		average: -148.16
Episode:     7		score: -155.93		average: -149.27
Episode:     8		score:  -88.87		average: -141.72
Episode:     9		score:  -77.70		average: -134.60
Episode:    10		score: -123.55		average: -133.50
Episode:    11		score: -126.62		average: -132.87
Episode:    12		score: -132.57		average: -132.85
Episode:    13		score: -132.74		average: -132.84
Episode:    14		score: -122.05		average: -132.07
Episode:    15		score: -138.04		average: -132.47
Episode:    16		score:  -86.57		average: -129.60
Episode:    17		score:    6.76		average: -121.58
Episode:    18		score: -183.13		average: -125.00
Episode:    19		score: -224.71		avera

Episode:   163		score: -145.35		average: -139.04
Episode:   164		score: -141.13		average: -137.45
Episode:   165		score: -138.77		average: -137.21
Episode:   166		score: -172.82		average: -138.37
Episode:   167		score: -111.78		average: -137.18
Episode:   168		score: -100.62		average: -136.00
Episode:   169		score: -120.83		average: -136.78
Episode:   170		score: -107.73		average: -136.41
Episode:   171		score:  -95.80		average: -135.02
Episode:   172		score: -187.98		average: -135.43
Episode:   173		score:  -86.33		average: -132.69
Episode:   174		score:  -97.32		average: -130.89
Episode:   175		score: -120.05		average: -130.67
Episode:   176		score: -113.79		average: -130.16
Episode:   177		score: -157.90		average: -130.58
Episode:   178		score: -138.91		average: -130.73
Episode:   179		score: -121.33		average: -130.66
Episode:   180		score: -110.50		average: -130.70
Episode:   181		score: -161.10		average: -130.65
Episode:   182		score:  -90.48		average: -130.31
Episode:   183		scor

Episode:   331		score: -105.96		average: -129.33
Episode:   332		score: -157.47		average: -129.64
Episode:   333		score: -106.24		average: -129.45
Episode:   334		score: -201.73		average: -131.54
Episode:   335		score: -160.88		average: -132.20
Episode:   336		score: -179.96		average: -132.80
Episode:   337		score: -193.40		average: -133.79
Episode:   338		score: -120.74		average: -133.13
Episode:   339		score: -133.51		average: -133.39
Episode:   340		score: -145.69		average: -133.16
Episode:   341		score:   22.17		average: -131.37
Episode:   342		score: -151.41		average: -131.70
Episode:   343		score: -141.08		average: -131.33
Episode:   344		score: -122.43		average: -131.27
Episode:   345		score: -104.22		average: -131.23
Episode:   346		score: -159.66		average: -131.02
Episode:   347		score: -161.05		average: -131.58
Episode:   348		score: -154.07		average: -131.93
Episode:   349		score: -177.28		average: -132.59
Episode:   350		score: -127.66		average: -132.67
Episode:   351		scor

Episode:   499		score: -140.89		average: -134.68
Episode:   500		score: -105.07		average: -134.46
Episode:   501		score: -103.14		average: -134.07
Episode:   502		score:  -99.95		average: -134.05
Episode:   503		score:  -90.22		average: -133.71
Episode:   504		score: -105.92		average: -133.04
Episode:   505		score: -130.73		average: -132.71
Episode:   506		score: -137.20		average: -132.89
Episode:   507		score: -180.16		average: -133.36
Episode:   508		score: -131.59		average: -133.42
Episode:   509		score: -118.71		average: -133.22
Episode:   510		score:  -91.68		average: -132.54
Episode:   511		score: -127.55		average: -132.21
Episode:   512		score: -139.65		average: -132.53
Episode:   513		score:  -82.29		average: -131.53
Episode:   514		score: -108.99		average: -130.89
Episode:   515		score: -108.53		average: -130.06
Episode:   516		score: -108.18		average: -129.40
Episode:   517		score: -129.52		average: -129.17
Episode:   518		score: -139.21		average: -129.48
Episode:   519		scor

Episode:   667		score:  -10.94		average: -130.21
Episode:   668		score: -107.42		average: -129.67
Episode:   669		score: -100.27		average: -129.76
Episode:   670		score:  -96.69		average: -129.35
Episode:   671		score:   -5.55		average: -128.00
Episode:   672		score: -160.89		average: -128.61
Episode:   673		score: -134.66		average: -128.38
Episode:   674		score:  -36.86		average: -127.79
Episode:   675		score: -147.03		average: -127.54
Episode:   676		score: -122.28		average: -127.76
Episode:   677		score:  -11.77		average: -126.20
Episode:   678		score: -154.64		average: -126.33
Episode:   679		score: -133.11		average: -126.26
Episode:   680		score: -128.67		average: -125.93
Episode:   681		score:  -39.51		average: -125.07
Episode:   682		score: -142.39		average: -125.36
Episode:   683		score: -125.10		average: -125.43
Episode:   684		score: -125.47		average: -125.72
Episode:   685		score: -129.73		average: -125.65
Episode:   686		score: -150.66		average: -125.43
Episode:   687		scor

...

Episode:  2682		score: -134.69		average: -131.14
Episode:  2683		score: -138.01		average: -131.45
Episode:  2684		score: -141.37		average: -131.62
Episode:  2685		score: -132.42		average: -131.69
Episode:  2686		score: -133.99		average: -131.59
Episode:  2687		score:  -21.02		average: -131.65
Episode:  2688		score: -173.29		average: -131.63
Episode:  2689		score: -116.62		average: -131.37
Episode:  2690		score: -143.49		average: -131.32
Episode:  2691		score: -155.53		average: -131.45
Episode:  2692		score: -153.55		average: -131.69
Episode:  2693		score: -135.93		average: -131.45
Episode:  2694		score: -126.09		average: -131.13
Episode:  2695		score: -129.09		average: -131.79
Episode:  2696		score: -116.78		average: -131.19
Episode:  2697		score: -125.75		average: -130.71
Episode:  2698		score:  -19.72		average: -129.48
Episode:  2699		score: -123.25		average: -129.58
Episode:  2700		score: -168.03		average: -130.37
Episode:  2701		score: -144.53		average: -130.13
Episode:  2702		scor

Episode:  2851		score: -181.49		average: -133.69
Episode:  2852		score:  -92.54		average: -133.39
Episode:  2853		score: -125.86		average: -132.82
Episode:  2854		score: -147.43		average: -132.81
Episode:  2855		score: -151.71		average: -133.14
Episode:  2856		score: -141.75		average: -133.41
Episode:  2857		score:  -93.87		average: -132.98
Episode:  2858		score:  -86.68		average: -132.62
Episode:  2859		score: -141.13		average: -132.56
Episode:  2860		score:  -92.32		average: -132.04
Episode:  2861		score: -193.44		average: -134.13
Episode:  2862		score: -143.30		average: -134.18
Episode:  2863		score: -133.75		average: -133.96
Episode:  2864		score:  -88.26		average: -133.86
Episode:  2865		score: -166.52		average: -134.46
Episode:  2866		score: -126.32		average: -134.12
Episode:  2867		score: -100.15		average: -134.24
Episode:  2868		score: -138.92		average: -134.67
Episode:  2869		score: -115.78		average: -135.79
Episode:  2870		score: -145.44		average: -135.36
Episode:  2871		scor

Episode:  3019		score: -119.46		average: -134.72
Episode:  3020		score: -124.97		average: -134.73
Episode:  3021		score: -141.22		average: -134.84
Episode:  3022		score: -148.24		average: -134.79
Episode:  3023		score: -149.45		average: -134.90
Episode:  3024		score: -137.98		average: -135.09
Episode:  3025		score: -141.87		average: -135.09
Episode:  3026		score: -138.21		average: -135.46
Episode:  3027		score: -334.89		average: -137.58
Episode:  3028		score: -139.89		average: -137.63
Episode:  3029		score: -151.36		average: -137.93
Episode:  3030		score: -203.39		average: -138.61
Episode:  3031		score: -138.19		average: -139.05
Episode:  3032		score: -128.00		average: -139.25
Episode:  3033		score: -122.90		average: -139.21
Episode:  3034		score: -177.11		average: -139.72
Episode:  3035		score: -148.93		average: -139.97
Episode:  3036		score: -104.44		average: -139.05
Episode:  3037		score: -145.44		average: -139.43
Episode:  3038		score: -133.24		average: -139.51
Episode:  3039		scor

Episode:  3186		score: -152.08		average: -139.65
Episode:  3187		score: -155.16		average: -140.22
Episode:  3188		score: -163.40		average: -140.11
Episode:  3189		score: -205.32		average: -140.36
Episode:  3190		score:  -93.31		average: -139.66
Episode:  3191		score: -194.21		average: -140.40
Episode:  3192		score: -167.30		average: -140.88
Episode:  3193		score: -128.90		average: -140.88
Episode:  3194		score: -174.46		average: -141.12
Episode:  3195		score: -160.24		average: -141.56
Episode:  3196		score:  -89.25		average: -140.89
Episode:  3197		score: -139.20		average: -141.17
Episode:  3198		score: -134.55		average: -141.26
Episode:  3199		score: -141.19		average: -141.32
Episode:  3200		score:  -34.47		average: -139.95
Episode:  3201		score: -138.37		average: -138.77
Episode:  3202		score: -163.92		average: -138.82
Episode:  3203		score: -154.10		average: -138.60
Episode:  3204		score:   19.06		average: -136.81
Episode:  3205		score: -116.86		average: -136.25
Episode:  3206		scor

Episode:  3354		score: -111.65		average: -130.17
Episode:  3355		score: -135.27		average: -130.08
Episode:  3356		score: -112.04		average: -130.96
Episode:  3357		score: -117.52		average: -130.75
Episode:  3358		score: -120.12		average: -130.59
Episode:  3359		score:  -89.69		average: -129.66
Episode:  3360		score: -173.81		average: -130.18
Episode:  3361		score: -146.08		average: -130.20
Episode:  3362		score:   -5.04		average: -129.11
Episode:  3363		score: -147.99		average: -130.78
Episode:  3364		score: -150.68		average: -130.98
Episode:  3365		score: -166.55		average: -131.12
Episode:  3366		score: -190.26		average: -131.24
Episode:  3367		score: -105.21		average: -130.89
Episode:  3368		score: -175.03		average: -131.07
Episode:  3369		score: -151.39		average: -131.31
Episode:  3370		score: -190.04		average: -131.57
Episode:  3371		score: -164.03		average: -131.59
Episode:  3372		score: -133.02		average: -131.66
Episode:  3373		score: -117.18		average: -131.76
Episode:  3374		scor